In [25]:
import os
import numpy as np
import pandas as pd
from glob import glob
import cv2

In [26]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [27]:
# loading dataset

def load_data(path , split = 0.1):
    
    images = sorted(glob(os.path.join(path , "PNG/Original/*")))
    
    masks = sorted(glob(os.path.join(path , "PNG/Ground Truth/*")))
    
    total_size = len(images)
    valid_size = int(split * total_size)
    test_size = int(split * total_size)
    
    # 612 total 
    # 61 test
    # 61 validate
    
    train_x , valid_x = train_test_split(images , test_size = valid_size, random_state = 42)
    
    train_y , valid_y = train_test_split(masks , test_size = valid_size, random_state = 42)
    
    
    train_x , test_x = train_test_split(train_x , test_size = test_size, random_state = 42)
    
    train_y , test_y = train_test_split(train_y , test_size = test_size, random_state = 42)
    
    return (train_x , train_y ) , (valid_x , valid_y) , (test_x , test_y)

In [28]:
def read_image(path):
    
    path = path.decode()
    x = cv2.imread(path , cv2.IMREAD_COLOR)
    x = cv2.resize(x , (256, 256))
    x = x/255.0
    # size is 256, 256 , 3
    return x


In [29]:
def read_mask(path):
    
    path = path.decode()
    x = cv2.imread(path , cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x , (256, 256))
    x = x/255.0
    # size is 256, 256
    x = np.expand_dims(x, axis = -1)
    # size is 256, 256 , 1
    
    return x

In [30]:
def tf_parse (x,y) :
    
    def _parse(x,y):
        
        x = read_image(x)
        y = read_mask(y)
        
        return x,y
    x,y = tf.numpy_function(_parse , [x,y] , [tf.float64 , tf.float64])
    
    x.set_shape([256,256,3])
    y.set_shape([256,256,1])
    
    return x,y

In [31]:
def tf_dataset(x,y,batch = 8):
    
    dataset = tf.data.Dataset.from_tensor_slices((x,y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.repeat()
    return dataset

In [32]:
if __name__ == "__main__":
    
    path = "/kaggle/input/cvcclinicdb"
    (train_x , train_y ) , (valid_x , valid_y) , (test_x , test_y) = load_data(path,0.1)
    
    # 612 total 
    # 61 test
    # 61 validate
    # 490 train
    
    ds = tf_dataset(test_x , test_y)
    
    for x, y in ds:
        
        print(x.shape , y.shape) # batch of 8 images with 8 img and 8 masks
        break

(8, 256, 256, 3) (8, 256, 256, 1)


In [33]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, UpSampling2D, Concatenate, Input, ZeroPadding2D
from tensorflow.keras.models import Model

def batchnorm_relu(inputs):
    x = BatchNormalization()(inputs)
    x = Activation("relu")(x)
    return x

def residual_block(inputs, num_filters, strides=1):
    """ Convolutional Layer """
    x = batchnorm_relu(inputs)
    x = Conv2D(num_filters, 3, padding="same", strides=strides)(x)
    x = batchnorm_relu(x)
    x = Conv2D(num_filters, 3, padding="same", strides=1)(x)

    """ Shortcut Connection """
    s = Conv2D(num_filters, 1, padding="same", strides=strides)(inputs)
    x = x + s
    return x

def decoder_block(inputs, skip_features, num_filters):
    x = UpSampling2D((2, 2))(inputs)
    x = Concatenate()([x, skip_features])
    x = residual_block(x, num_filters, strides=1)
    return x

def build_resunet(input_shape):
    inputs = Input(input_shape)

    """ Encoder 1 """
    x = Conv2D(64, 3, padding="same", strides=1)(inputs)
    x = batchnorm_relu(x)
    x = Conv2D(64, 3, padding="same", strides=1)(x)
    s = Conv2D(64, 1, padding="same", strides=1)(inputs)
    s1 = x + s

    """ Encoder 2 and 3 """
    s2 = residual_block(s1, 128, strides=2)
    s3 = residual_block(s2, 256, strides=2)

    """ Bridge """
    b = residual_block(s3, 512, strides=2)

    """ Decoder 1, 2, 3 """
    d1 = decoder_block(b, s3, 256)
    d2 = decoder_block(d1, s2, 128)
    d3 = decoder_block(d2, s1, 64)

    """ Classifier """
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d3)

    """ Model """
    model = Model(inputs, outputs)
    return model

if __name__ == "__main__":
    model = build_resunet((256, 256, 3))
    model.summary()


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_44 (Conv2D)             (None, 256, 256, 64  1792        ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_26 (BatchN  (None, 256, 256, 64  256        ['conv2d_44[0][0]']              
 ormalization)                  )                                                           

In [34]:

import tensorflow.keras.backend as K

from sklearn.metrics import jaccard_score,confusion_matrix

def IoU_coef(y_true, y_pred):
    
    print(y_true.shape)
    print(y_pred.shape)
    
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)

def dice_coef(y_true, y_pred):
    
    print(y_true.shape)
    print(y_pred.shape)
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2.0 * intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1.0)

In [35]:
from tensorflow import keras
import tensorflow.keras.backend as K
import numpy as np
import tensorflow as tf


In [36]:
# Cross Entropy
# Focal 
# Dice 
# Tversky
# Tversky Focal
# Combo
# Unified Focal (Sym)
# Unified Focal (Asym)


In [37]:
# dice_loss = DiceLoss()
# binary_focal_loss = BinaryFocalLoss()
# combo_loss = binary_crossentropy + dice_loss


In [38]:

def dice_coef(y_true, y_pred, smooth=100):        
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    dice = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return dice

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)


In [39]:
def sym_unified_focal_loss(y_true, y_pred ):
    
    axis = identify_axis(y_true.get_shape())
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    
    def loss_function(y_true,y_pred):
        weight=0.5
        delta=0.6 
        gamma=0.5
        symmetric_ftl = symmetric_focal_tversky_loss(delta=delta, gamma=gamma)(y_true,y_pred)
        symmetric_fl = symmetric_focal_loss(delta=delta, gamma=gamma)(y_true,y_pred)
        if weight is not None:
            return (weight * symmetric_ftl) + ((1-weight) * symmetric_fl)
        else:
            return symmetric_ftl + symmetric_fl

    return loss_function(y_true,y_pred)

In [40]:
def iou(y_true, y_pred) :
    
    def f (y_true, y_pred) :
        
        y_true_f = K.flatten(y_true)
        y_pred_f = K.flatten(y_pred)
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    
    return tf.numpy_function(f, [y_true , y_pred] , tf.float32)
        
      

In [41]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint , ReduceLROnPlateau , CSVLogger, TensorBoard
from tensorflow.keras.metrics import Recall , Precision
from keras import optimizers
from tensorflow.keras.losses import BinaryFocalCrossentropy
  
        
if __name__ == "__main__":
    
    
    np.random.seed(42)
    tf.random.set_seed(42)
    batch = 16
    lr = 1e-6
    epochs = 100
    
    train_dataset= tf_dataset(train_x , train_y, batch = batch)
    valid_dataset= tf_dataset(valid_x , valid_y, batch = batch)
    
    
    model = build_resunet((256, 256, 3))
    
    binary_cross_entropy = tf.keras.losses.BinaryCrossentropy()
    opt = tf.keras.optimizers.Adam(lr)
    
    metrics = ["acc" , Recall() , Precision() , iou , dice_coef]
    
    
    model.compile(loss = binary_cross_entropy , optimizer = opt , metrics = metrics)
    
    callbacks  = [
        ModelCheckpoint("/kaggle/working/model.h5"),
        ReduceLROnPlateau(monitor = "val_loss" , factor = 0.5 , patience = 5),
        EarlyStopping(monitor = "val_loss" , patience = 30 , restore_best_weights = False)
    ]
    
    
    train_steps = len(train_x) // batch
    valid_steps = len(valid_x) // batch
    
    if len(train_x) % batch != 0 :
        train_steps+= 1
        
    if len(valid_x) % batch != 0 :
        valid_steps+= 1


    model.fit(
        
        train_dataset, 
        validation_data = valid_dataset,
        epochs = epochs,
        steps_per_epoch = train_steps,
        validation_steps = valid_steps,
        callbacks = callbacks,
        shuffle = False
    )
    
    

Epoch 1/100
31/31 [==============================] - 29s 555ms/step - loss: 1.0866 - acc: 0.5335 - recall_1: 0.0915 - precision_1: 0.0227 - iou: 0.0281 - dice_coef: 0.0548 - val_loss: 0.6403 - val_acc: 0.8620 - val_recall_1: 0.0717 - val_precision_1: 0.1002 - val_iou: 0.0753 - val_dice_coef: 0.1400 - lr: 1.0000e-06
Epoch 2/100
31/31 [==============================] - 17s 552ms/step - loss: 0.5097 - acc: 0.8495 - recall_1: 0.0524 - precision_1: 0.0783 - iou: 0.0453 - dice_coef: 0.0868 - val_loss: 0.5258 - val_acc: 0.9074 - val_recall_1: 0.0012 - val_precision_1: 0.0211 - val_iou: 0.0737 - val_dice_coef: 0.1373 - lr: 1.0000e-06
Epoch 3/100
31/31 [==============================] - 17s 555ms/step - loss: 0.3823 - acc: 0.8844 - recall_1: 0.0885 - precision_1: 0.2324 - iou: 0.0756 - dice_coef: 0.1405 - val_loss: 0.4286 - val_acc: 0.9095 - val_recall_1: 2.7695e-04 - val_precision_1: 0.0083 - val_iou: 0.0721 - val_dice_coef: 0.1345 - lr: 1.0000e-06
Epoch 4/100
31/31 [==========================

In [42]:
def read_image_new(path):
    
    x = cv2.imread(path , cv2.IMREAD_COLOR)
    x = cv2.resize(x , (256, 256))
    x = x/255.0
    # size is 256, 256 , 3
    return x


def read_mask_new(path):

    x = cv2.imread(path , cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x , (256, 256))
    # size is 256, 256
    x = np.expand_dims(x, axis = -1)
    # size is 256, 256 , 1
    
    return x

def mask_parse(mask):
    
    mask = np.squeeze(mask)
    mask = [mask , mask , mask]
    
    mask = np.transpose(mask , (1,2,0))
    return mask

In [43]:
from tqdm import tqdm
from tensorflow.keras.utils import CustomObjectScope



In [44]:
if __name__ == "__main__":
    
    test_dataset = tf_dataset(test_x , test_y , batch = batch)
    test_steps = len(test_x) // batch
    
    if len(test_x)%batch != 0 :
        test_steps +=1
        
    with CustomObjectScope({'iou' : iou , 'dice_coef' : dice_coef ,  'dice_coef_loss' :dice_coef_loss }):
    
        model = tf.keras.models.load_model("/kaggle/working/model.h5")
        
    model.evaluate(test_dataset , steps = test_steps)
    
    for i , (x,y) in tqdm(enumerate(zip(test_x , test_y)) , total= len(test_x)):
        
        x = read_image_new(x)
        y = read_mask_new(y)
        
        y_pred = model.predict(np.expand_dims(x , axis = 0))
        
        y_pred= y_pred[0] > 0.5
        
        h,w, _ = x.shape
        
        white_line = np.ones((h,10,3)) * 255.0
        
        all_images = [
            
            x*255.0, white_line,
            
            mask_parse(y) , white_line,
            mask_parse(y_pred)*255.0
            
        ]
        
        image = np.concatenate (all_images, axis =1)
        cv2.imwrite(f"/kaggle/working/{i}.png" , image)


4/4 [==============================] - 1s 175ms/step - loss: 0.1753 - acc: 0.9312 - recall_1: 0.4838 - precision_1: 0.7317 - iou: 0.3274 - dice_coef: 0.4933


  0%|          | 0/61 [00:00<?, ?it/s]

1/1 [==============================] - 0s 312ms/step


  2%|▏         | 1/61 [00:00<00:22,  2.62it/s]

1/1 [==============================] - 0s 19ms/step


...

 93%|█████████▎| 57/61 [00:04<00:00, 12.61it/s]

1/1 [==============================] - 0s 19ms/step


 97%|█████████▋| 59/61 [00:05<00:00, 12.64it/s]

1/1 [==============================] - 0s 19ms/step


100%|██████████| 61/61 [00:05<00:00, 11.63it/s]


In [45]:
y_pred.shape

(256, 256, 1)

In [46]:
# !zip -r segmentation_model.zip  "/kaggle/input/segmentation-model"

In [48]:
!zip -r cvc_new_new.zip   "/kaggle/working/"

  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/10.png (deflated 6%)
  adding: kaggle/working/18.png (deflated 7%)
  adding: kaggle/working/45.png (deflated 5%)
  adding: kaggle/working/16.png (deflated 7%)
  adding: kaggle/working/38.png (deflated 7%)
  adding: kaggle/working/27.png (deflated 7%)
  adding: kaggle/working/32.png (deflated 9%)
  adding: kaggle/working/59.png (deflated 5%)
  adding: kaggle/working/7.png (deflated 6%)
  adding: kaggle/working/0.png (deflated 6%)
  adding: kaggle/working/cvc_new.zip (stored 0%)
  adding: kaggle/working/49.png (deflated 7%)
  adding: kaggle/working/1.png (deflated 7%)
  adding: kaggle/working/50.png (deflated 7%)
  adding: kaggle/working/52.png (deflated 6%)
  adding: kaggle/working/5.png (deflated 6%)
  adding: kaggle/working/29.png (deflated 8%)
  adding: kaggle/working/44.png (deflated 5%)
  adding: kaggle/working/11.png (deflated 7%)
  adding: kaggle/working/35.png (deflated 5%)
  adding: kaggle/working/9.png (deflated 7%